In [1]:
import gpflow
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd


%matplotlib inline

from gpflow import set_trainable
from gpflow.utilities import ops, print_summary
from gpflow.ci_utils import ci_niter
from gpflow.config import set_default_float, default_float, set_default_summary_fmt

f64 = gpflow.utilities.to_default_float

plt.rcParams["figure.figsize"] = (12, 6)
np.random.seed(123)

In [2]:
import yfinance as yf
stocks="ESS,GRMN,SCHW,SPG,CF,ACN,LB,ADP,TROW,CTSH,FIS,COP,INTC,NFX,BF-B,TDG,MCHP,D,SYY,MTD,FITB,COL,PVH,JBHT,DAL,UNH,AON,CRM,SLB,XEC,XRAY,GPN,ALGN,LNC,ADM,KSU,YUM,AEP,GPS,LOW,DLR,ADS,ADSK,STT,AME,CAG,HRL,ESRX,XOM,NSC,A,PH,AMD,BBY"
stocks=stocks.replace(",", " ")
yf_stocks = yf.download(stocks, start="2005-01-01", end="2021-01-01")

[*********************100%***********************]  54 of 54 completed


In [3]:
daily_rets = yf_stocks['Adj Close'].pct_change()
len(daily_rets)

4028

In [4]:
daily_rets = daily_rets[:][1:]
len(daily_rets)

4027

In [5]:
test_r = daily_rets[:][:]
df_r = pd.DataFrame(test_r)
df_t = df_r.T
df = df_t.dropna()
df_train = df.iloc[:50, :10]
df_train.shape

(50, 10)

In [6]:
r = tf.convert_to_tensor(df_train, dtype=default_float())
print("Number of Stocks in Portfolio: {} and Number of Days: {}".format(r.shape[0], r.shape[1]))

Number of Stocks in Portfolio: 50 and Number of Days: 10


In [7]:
latent_dim = 4  # number of latent dimensions
num_inducing = 50  # number of inducing pts
num_data = r.shape[0]
print(num_data)

50


In [8]:
X_mean_init = ops.pca_reduce(r, latent_dim)
X_var_init = tf.ones((num_data, latent_dim), dtype=default_float())

In [9]:
np.random.seed(1)  # for reproducibility
inducing_variable = tf.convert_to_tensor(
    np.random.permutation(X_mean_init.numpy())[:num_inducing], dtype=default_float()
)

In [10]:
# Create multi-output kernel from kernel list
lengthscales = tf.convert_to_tensor([1.0] * latent_dim, dtype=default_float())
kernel = gpflow.kernels.RBF(lengthscales=lengthscales)

In [11]:
gplvm = gpflow.models.BayesianGPLVM(
    r,
    X_data_mean=X_mean_init,
    X_data_var=X_var_init,
    kernel=kernel,
    inducing_variable=inducing_variable,
)
print_summary(gplvm)

╒══════════════════════════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════╤═════════╤═══════════════════════════════════════════════════════╕
│ name                                         │ class     │ transform        │ prior   │ trainable   │ shape   │ dtype   │ value                                                 │
╞══════════════════════════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════╪═════════╪═══════════════════════════════════════════════════════╡
│ BayesianGPLVM.kernel.kernels[0].variance     │ Parameter │ Softplus         │         │ True        │ ()      │ float64 │ 1.0                                                   │
├──────────────────────────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼───────────────────────────────────────────────────────┤
│ BayesianGPLVM.kernel.kernels[0].lengthscales │ Parameter │ Softplus         │         │ True      

In [12]:
gplvm.likelihood.variance.assign(0.01)
gplvm.kernel.kernels[0].variance.prior = tfd.InverseGamma(f64(3.0),f64(1.0))
gplvm.kernel.kernels[0].lengthscales.prior = tfd.InverseGamma(f64(3.0),f64(1.0))
gplvm.kernel.kernels[1].variance.prior = tfd.Normal(0,0.5)
print_summary(gplvm)

╒══════════════════════════════════════════════╤═══════════╤══════════════════╤══════════════╤═════════════╤═════════╤═════════╤═══════════════════════════════════════════════════════╕
│ name                                         │ class     │ transform        │ prior        │ trainable   │ shape   │ dtype   │ value                                                 │
╞══════════════════════════════════════════════╪═══════════╪══════════════════╪══════════════╪═════════════╪═════════╪═════════╪═══════════════════════════════════════════════════════╡
│ BayesianGPLVM.kernel.kernels[0].variance     │ Parameter │ Softplus         │ InverseGamma │ True        │ ()      │ float64 │ 1.0                                                   │
├──────────────────────────────────────────────┼───────────┼──────────────────┼──────────────┼─────────────┼─────────┼─────────┼───────────────────────────────────────────────────────┤
│ BayesianGPLVM.kernel.kernels[0].lengthscales │ Parameter │ Softplus      

In [ ]:
set_trainable(gplvm.inducing_variable, False)

In [ ]:
opt = gpflow.optimizers.Scipy()
maxiter = ci_niter(1000)
_ = opt.minimize(
    gplvm.training_loss,
    variables=gplvm.trainable_variables,
    options=dict(maxiter=maxiter),
)

In [ ]:
print_summary(gplvm)

In [ ]:
X_pca = ops.pca_reduce(r, latent_dim).numpy()
gplvm_X_mean = gplvm.X_data_mean.numpy()

f, ax = plt.subplots(1, 2, figsize=(10, 6))

ax[0].scatter(X_pca[:, 0], X_pca[:, 1])
ax[1].scatter(gplvm_X_mean[:, 0], gplvm_X_mean[:, 1])
ax[0].set_title("PCA")
ax[1].set_title("Bayesian GPLVM")

In [ ]:
set_trainable(gplvm.inducing_variable.Z, False)
set_trainable(gplvm.likelihood.variance, False)
set_trainable(gplvm.X_data_var, False)
print_summary(gplvm)

In [ ]:
num_burnin_steps = ci_niter(100)
num_samples = ci_niter(500)

# Note that here we need model.trainable_parameters, not trainable_variables - only parameters can have priors!
hmc_helper = gpflow.optimizers.SamplingHelper(
    gplvm.log_posterior_density, gplvm.trainable_parameters
)

hmc = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=hmc_helper.target_log_prob_fn, num_leapfrog_steps=10, step_size=0.01
)

adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(
    hmc, num_adaptation_steps=10, target_accept_prob=f64(0.75), adaptation_rate=0.1
)


@tf.function
def run_chain_fn():
    return tfp.mcmc.sample_chain(
        num_results=num_samples,
        num_burnin_steps=num_burnin_steps,
        current_state=hmc_helper.current_state,
        kernel=adaptive_hmc,
        trace_fn=lambda _, pkr: pkr.inner_results.is_accepted,
    )


samples, _ = run_chain_fn()
constrained_samples = hmc_helper.convert_to_constrained_values(samples)

In [ ]:
print_summary(gplvm)

In [ ]:
X_pca = ops.pca_reduce(r, latent_dim).numpy()
gplvm_X_mean = gplvm.X_data_mean.numpy()

f, ax = plt.subplots(1, 2, figsize=(10, 6))

ax[0].scatter(X_pca[:, 0], X_pca[:, 1])
ax[1].scatter(gplvm_X_mean[:, 0], gplvm_X_mean[:, 1])
ax[0].set_title("PCA")
ax[1].set_title("Bayesian GPLVM")

In [ ]:
param_to_name = {param: name for name, param in gpflow.utilities.parameter_dict(gplvm).items()}

def marginal_samples(samples, parameters, y_axis_label):
    fig, axes = plt.subplots(1, len(param_to_name), figsize=(15, 3), constrained_layout=True)
    for ax, val, param in zip(axes, samples, parameters):
        ax.hist(np.stack(val).flatten(), bins=20)
        ax.set_title(param_to_name[param])
    fig.suptitle(y_axis_label)
    plt.show()


marginal_samples(samples, gplvm.trainable_parameters, "unconstrained variable samples")
marginal_samples(constrained_samples, gplvm.trainable_parameters, "constrained parameter samples")

In [ ]:
K = kernel.K(gplvm.X_data_mean.numpy())

In [ ]:
plt.matshow(K)
plt.show()